## Preprocessing

In [29]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from keras.callbacks import ModelCheckpoint

# Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [30]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
drop_columns = ['EIN', 'NAME']
application_df = application_df.drop(columns=drop_columns)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [31]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [32]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [33]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_constant_high = 700
cutoff_constant_low = 20

low_bin = []
mid_bin = []
for item in application_df['APPLICATION_TYPE'].value_counts().items():
    if item[1] < cutoff_constant_high and item[1] > cutoff_constant_low:
        mid_bin.append(item[0])
    elif item[1] < cutoff_constant_low:
        low_bin.append(item[0])

# Replace in dataframe
for app in low_bin:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"LowOther")
for app in mid_bin:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"MidOther")
    
# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3          27037
T4           1542
T6           1216
T5           1173
T19          1065
MidOther      777
T8            737
T7            725
LowOther       27
Name: APPLICATION_TYPE, dtype: int64

In [34]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [35]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x : x>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [36]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_constant_high = 500
cutoff_constant_low = 2

low_bin = []
mid_bin = []

for item in application_df['CLASSIFICATION'].value_counts().items():
    if item[1] < cutoff_constant_high and item[1] >= cutoff_constant_low:
        mid_bin.append(item[0])
    elif item[1] < cutoff_constant_low:
        low_bin.append(item[0])

# Replace in dataframe
for cls in low_bin:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"LowOther")
for cls in mid_bin:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"MidOther")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000       17326
C2000        6074
C1200        4837
C3000        1918
C2100        1883
MidOther     1458
C7000         777
LowOther       26
Name: CLASSIFICATION, dtype: int64

In [37]:
# Convert categorical data to numeric with `pd.get_dummies`
converted_data = pd.get_dummies(data=application_df, columns=list(set(['APPLICATION_TYPE', 'CLASSIFICATION', 'AFFILIATION',
                                                             'USE_CASE', 'ORGANIZATION', 'SPECIAL_CONSIDERATIONS',
                                                             'INCOME_AMT'])-set(drop_columns)))

In [38]:
converted_data

,STATUS,ASK_AMT,IS_SUCCESSFUL,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,...,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,1,108590,1,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,1,5000,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
3,1,6692,1,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
34295,1,5000,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
34296,1,5000,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
34297,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [39]:
converted_data.shape

(34299, 46)

In [40]:
# Split our preprocessed data into our features and target arrays
# y is target
# x is features
y = converted_data['IS_SUCCESSFUL']
X = converted_data.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.25)

In [41]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [42]:
X_test.shape

(8575, 45)

## Compile, Train and Evaluate the Model

In [43]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = Sequential()

# First hidden layer
nn.add(Dense(units=90, activation = 'relu', input_dim=X_test.shape[1]))

# Second hidden layer
nn.add(Dense(units=20, activation='relu'))

# Output layer
nn.add(Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 90)                4140      
                                                                 
 dense_4 (Dense)             (None, 20)                1820      
                                                                 
 dense_5 (Dense)             (None, 1)                 21        
                                                                 
Total params: 5981 (23.36 KB)
Trainable params: 5981 (23.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [44]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [45]:
# Create a checkpoint to save the best output
checkpoint = ModelCheckpoint("best_model_checkpoint.h5", monitor='accuracy', verbose=1,
    save_best_only=True, mode='auto', save_freq=5)

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=40, callbacks=[checkpoint])

Epoch 1/40
  1/804 [..............................] - ETA: 1:42 - loss: 0.7059 - accuracy: 0.4688
Epoch 1: accuracy improved from -inf to 0.51250, saving model to best_model_checkpoint.h5

Epoch 1: accuracy improved from 0.51250 to 0.60938, saving model to best_model_checkpoint.h5

Epoch 1: accuracy did not improve from 0.60938

Epoch 1: accuracy improved from 0.60938 to 0.63437, saving model to best_model_checkpoint.h5

Epoch 1: accuracy improved from 0.63437 to 0.64375, saving model to best_model_checkpoint.h5

Epoch 1: accuracy improved from 0.64375 to 0.64688, saving model to best_model_checkpoint.h5

Epoch 1: accuracy improved from 0.64688 to 0.65804, saving model to best_model_checkpoint.h5
 35/804 [>.............................] - ETA: 1s - loss: 0.6688 - accuracy: 0.6580  
Epoch 1: accuracy did not improve from 0.65804

Epoch 1: accuracy did not improve from 0.65804

Epoch 1: accuracy improved from 0.65804 to 0.66500, saving model to best_model_checkpoint.h5

Epoch 1: accuracy

/Users/owner/anaconda3/envs/dev/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 1: accuracy improved from 0.70101 to 0.70265, saving model to best_model_checkpoint.h5
165/804 [=====>........................] - ETA: 0s - loss: 0.6061 - accuracy: 0.7027
Epoch 1: accuracy improved from 0.70265 to 0.70331, saving model to best_model_checkpoint.h5

Epoch 1: accuracy did not improve from 0.70331

Epoch 1: accuracy did not improve from 0.70331

Epoch 1: accuracy did not improve from 0.70331

Epoch 1: accuracy did not improve from 0.70331

Epoch 1: accuracy did not improve from 0.70331

Epoch 1: accuracy did not improve from 0.70331

Epoch 1: accuracy did not improve from 0.70331

Epoch 1: accuracy improved from 0.70331 to 0.70342, saving model to best_model_checkpoint.h5

Epoch 1: accuracy improved from 0.70342 to 0.70465, saving model to best_model_checkpoint.h5

Epoch 1: accuracy improved from 0.70465 to 0.70653, saving model to best_model_checkpoint.h5
220/804 [=======>......................] - ETA: 0s - loss: 0.5966 - accuracy: 0.7065
Epoch 1: accuracy improve


Epoch 1: accuracy did not improve from 0.71863

Epoch 1: accuracy did not improve from 0.71863

Epoch 1: accuracy did not improve from 0.71863

Epoch 1: accuracy did not improve from 0.71863

Epoch 1: accuracy did not improve from 0.71863
772/804 [===========================>..] - ETA: 0s - loss: 0.5740 - accuracy: 0.7164
Epoch 1: accuracy did not improve from 0.71863

Epoch 1: accuracy did not improve from 0.71863

Epoch 1: accuracy did not improve from 0.71863

Epoch 1: accuracy did not improve from 0.71863

Epoch 1: accuracy did not improve from 0.71863

Epoch 1: accuracy did not improve from 0.71863
804/804 [==============================] - 1s 930us/step - loss: 0.5722 - accuracy: 0.7176
Epoch 2/40

Epoch 2: accuracy improved from 0.71863 to 0.71875, saving model to best_model_checkpoint.h5
  1/804 [..............................] - ETA: 4s - loss: 0.6725 - accuracy: 0.7188
Epoch 2: accuracy improved from 0.71875 to 0.75000, saving model to best_model_checkpoint.h5

Epoch 2: accu

699/804 [=========================>....] - ETA: 0s - loss: 0.5535 - accuracy: 0.7306
Epoch 2: accuracy did not improve from 0.75000

Epoch 2: accuracy did not improve from 0.75000

Epoch 2: accuracy did not improve from 0.75000

Epoch 2: accuracy did not improve from 0.75000

Epoch 2: accuracy did not improve from 0.75000

Epoch 2: accuracy did not improve from 0.75000

Epoch 2: accuracy did not improve from 0.75000

Epoch 2: accuracy did not improve from 0.75000

Epoch 2: accuracy did not improve from 0.75000

Epoch 2: accuracy did not improve from 0.75000

Epoch 2: accuracy did not improve from 0.75000

Epoch 2: accuracy did not improve from 0.75000

Epoch 2: accuracy did not improve from 0.75000

Epoch 2: accuracy did not improve from 0.75000

Epoch 2: accuracy did not improve from 0.75000

Epoch 2: accuracy did not improve from 0.75000

Epoch 2: accuracy did not improve from 0.75000

Epoch 2: accuracy did not improve from 0.75000
786/804 [============================>.] - ETA: 0s -

...


Epoch 32: accuracy did not improve from 0.84375

Epoch 32: accuracy did not improve from 0.84375

Epoch 32: accuracy did not improve from 0.84375

Epoch 32: accuracy did not improve from 0.84375

Epoch 32: accuracy did not improve from 0.84375

Epoch 32: accuracy did not improve from 0.84375

Epoch 32: accuracy did not improve from 0.84375

Epoch 32: accuracy did not improve from 0.84375

Epoch 32: accuracy did not improve from 0.84375
804/804 [==============================] - 0s 560us/step - loss: 0.5399 - accuracy: 0.7375
Epoch 33/40
  1/804 [..............................] - ETA: 0s - loss: 0.6636 - accuracy: 0.6562
Epoch 33: accuracy did not improve from 0.84375

Epoch 33: accuracy did not improve from 0.84375

Epoch 33: accuracy did not improve from 0.84375

Epoch 33: accuracy did not improve from 0.84375

Epoch 33: accuracy did not improve from 0.84375

Epoch 33: accuracy did not improve from 0.84375

Epoch 33: accuracy did not improve from 0.84375

Epoch 33: accuracy did not i


Epoch 33: accuracy did not improve from 0.84375
719/804 [=========================>....] - ETA: 0s - loss: 0.5415 - accuracy: 0.7365
Epoch 33: accuracy did not improve from 0.84375

Epoch 33: accuracy did not improve from 0.84375

Epoch 33: accuracy did not improve from 0.84375

Epoch 33: accuracy did not improve from 0.84375

Epoch 33: accuracy did not improve from 0.84375

Epoch 33: accuracy did not improve from 0.84375

Epoch 33: accuracy did not improve from 0.84375

Epoch 33: accuracy did not improve from 0.84375

Epoch 33: accuracy did not improve from 0.84375

Epoch 33: accuracy did not improve from 0.84375

Epoch 33: accuracy did not improve from 0.84375

Epoch 33: accuracy did not improve from 0.84375

Epoch 33: accuracy did not improve from 0.84375

Epoch 33: accuracy did not improve from 0.84375

Epoch 33: accuracy did not improve from 0.84375

Epoch 33: accuracy did not improve from 0.84375

Epoch 33: accuracy did not improve from 0.84375
804/804 [=========================


Epoch 34: accuracy did not improve from 0.84375

Epoch 34: accuracy did not improve from 0.84375

Epoch 34: accuracy did not improve from 0.84375

Epoch 34: accuracy did not improve from 0.84375

Epoch 34: accuracy did not improve from 0.84375

Epoch 34: accuracy did not improve from 0.84375

Epoch 34: accuracy did not improve from 0.84375

Epoch 34: accuracy did not improve from 0.84375

Epoch 34: accuracy did not improve from 0.84375

Epoch 34: accuracy did not improve from 0.84375
709/804 [=========================>....] - ETA: 0s - loss: 0.5398 - accuracy: 0.7366
Epoch 34: accuracy did not improve from 0.84375

Epoch 34: accuracy did not improve from 0.84375

Epoch 34: accuracy did not improve from 0.84375

Epoch 34: accuracy did not improve from 0.84375

Epoch 34: accuracy did not improve from 0.84375

Epoch 34: accuracy did not improve from 0.84375

Epoch 34: accuracy did not improve from 0.84375

Epoch 34: accuracy did not improve from 0.84375

Epoch 34: accuracy did not improv


Epoch 35: accuracy did not improve from 0.84375

Epoch 35: accuracy did not improve from 0.84375

Epoch 35: accuracy did not improve from 0.84375

Epoch 35: accuracy did not improve from 0.84375

Epoch 35: accuracy did not improve from 0.84375
629/804 [======================>.......] - ETA: 0s - loss: 0.5407 - accuracy: 0.7338
Epoch 35: accuracy did not improve from 0.84375

Epoch 35: accuracy did not improve from 0.84375

Epoch 35: accuracy did not improve from 0.84375

Epoch 35: accuracy did not improve from 0.84375

Epoch 35: accuracy did not improve from 0.84375

Epoch 35: accuracy did not improve from 0.84375

Epoch 35: accuracy did not improve from 0.84375

Epoch 35: accuracy did not improve from 0.84375

Epoch 35: accuracy did not improve from 0.84375

Epoch 35: accuracy did not improve from 0.84375

Epoch 35: accuracy did not improve from 0.84375

Epoch 35: accuracy did not improve from 0.84375

Epoch 35: accuracy did not improve from 0.84375

Epoch 35: accuracy did not improv


Epoch 36: accuracy did not improve from 0.84375

Epoch 36: accuracy did not improve from 0.84375

Epoch 36: accuracy did not improve from 0.84375

Epoch 36: accuracy did not improve from 0.84375

Epoch 36: accuracy did not improve from 0.84375

Epoch 36: accuracy did not improve from 0.84375

Epoch 36: accuracy did not improve from 0.84375

Epoch 36: accuracy did not improve from 0.84375

Epoch 36: accuracy did not improve from 0.84375

Epoch 36: accuracy did not improve from 0.84375

Epoch 36: accuracy did not improve from 0.84375

Epoch 36: accuracy did not improve from 0.84375

Epoch 36: accuracy did not improve from 0.84375

Epoch 36: accuracy did not improve from 0.84375

Epoch 36: accuracy did not improve from 0.84375
628/804 [======================>.......] - ETA: 0s - loss: 0.5376 - accuracy: 0.7397
Epoch 36: accuracy did not improve from 0.84375

Epoch 36: accuracy did not improve from 0.84375

Epoch 36: accuracy did not improve from 0.84375

Epoch 36: accuracy did not improv

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity.h5')